In [ ]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from mpi4py import MPI
import matplotlib.pyplot as plt
from vpm_py import VPM
from vpm_py.console_io import print_IMPORTANT,print_red
from test_problems.oseen_vortex import oseen_assign
from vpm_py.visualization import StandardVisualizer

In [ ]:
def initialize_oseen_vortex(
    vpm: VPM, 
    gamma: float, 
    viscosity: float,
    density: float,
    t: float 
):
    # Create particles
    NVR = 100
    neq = vpm.num_equations

    XPR_zero = np.zeros((3, NVR), dtype=np.float64)
    XPR_zero[:, 0] = [-0.5 , -0.5 , -2.]
    XPR_zero[:, 1] = [ 0.5 ,  0.5 ,  2.]
    QPR_zero = np.ones((neq + 1, NVR), dtype=np.float64)

    # Initialization VPM
    vpm.vpm_define(
        num_equations=neq,
        particle_positions= XPR_zero, 
        particle_charges= QPR_zero, 
    )
    
    # Initialize Hill Vortex
    st = MPI.Wtime()
    print_IMPORTANT("Hill vortex initialization")
    velocity_oseen, vorticity_oseen, pressure_oseen = oseen_assign(
        vpm = vpm,
        viscosity= viscosity,
        density= density,
        t = t, 
        gamma= gamma
    )
    vpm.particle_mesh.set_rhs_pm(vorticity_oseen)
    print_red("Setting RHS_pm as computed from the hill vortex")
    
    st = MPI.Wtime()
    print_red("Remeshing")
    XPR_hill, QPR_hill = vpm.remesh_particles(project_particles=False, cut_off=1e-8) 
    et = MPI.Wtime()
    print(f"\tRemeshing finished in {int((et - st) / 60)}m {(et - st) % 60:.2f}s\n")

    print_IMPORTANT("Particles initialized")
    NVR = vpm.particles.NVR
    return XPR_hill, QPR_hill, NVR, velocity_oseen, vorticity_oseen, pressure_oseen

In [ ]:
REYNOLDS_NUMBER=10
REMESH_FREQUENCY=40
apply_vorticity_correction=False

In [ ]:
# PROBLEM STATEMENT
UINF = np.array([0.0, 0.0, 1.0])
gamma = 2.0
# Reynolds number = U * L / nu , where U is the velocity, L is the radius of the sphere and nu is the kinematic viscosity
# nu = U * L / REYNOLDS_NUMBER
VISCOSITY = 10e-3 
DENSITY = 1000.
# DT should be set according to the CFL condition: CFL = U * DT / dx < 1
dpm = np.array([0.1, 0.1, 0.1])
# dpm = np.array([0.02, 0.02, 0.02])
# dpm = np.array([0.04, 0.04, 0.04])

# CASE FOLDER
CASE_FOLDER = "/mnt/c/Users/tryfonas/Data/hill_vortex_5_seconds/oseen_pressure"

In [ ]:
# Initialize MPI
comm = MPI.COMM_WORLD
start_time = MPI.Wtime()
rank = comm.Get_rank()
np_procs = comm.Get_size()
neq = 3
# Initialize VPM
vpm = VPM(
    number_of_equations=3,
    number_of_processors=np_procs,
    rank=rank,
    verbocity=0,
    dx_particle_mesh=dpm[0],
    dy_particle_mesh=dpm[1],
    dz_particle_mesh=dpm[2],
    case_folder=CASE_FOLDER,
)
# plotter = StandardVisualizer(
#     plot_particles=("charge", "magnitude"),  # plot_mesh=("velocity", "magnitude")
# )
# vpm.attach_visualizer(plotter)

In [ ]:
(
    XPR_zero, QPR_zero, NVR, velocity_oseen, vorticity_oseen, pressure_oseen
) = initialize_oseen_vortex(
    vpm= vpm,
    gamma= gamma,
    viscosity= VISCOSITY,
    density= DENSITY,
    t=1
)
# Create the plot to live update the particles
vpm.update_plot(
    f"Time: {0:.2f}s",
)

In [ ]:
vpm.vpm_solve_velocity_deformation(
    timestep=0,
    num_equations=neq,
    particle_positions    =  XPR_zero,
    particle_charges      =  QPR_zero,
)
vpm.vpm_solve_pressure(density= DENSITY, viscosity= 0)

In [ ]:
U_PM = vpm.particle_mesh.velocity.copy()
# Convert U_PM to c order
U_PM = np.ascontiguousarray(U_PM, dtype=np.float64)

U_MESH = np.nan_to_num(U_PM, posinf=0.0, neginf=0.0)[0, :, :, :]
U_OSEEN = np.nan_to_num(velocity_oseen, posinf=0.0, neginf=0.0)[0, :, :, :]

# Get the error between the computed pressure and the hill vortex pressure
error = np.abs(U_MESH - U_OSEEN) 
print(f"Error: {np.max(error)}")
print(f"Error: {np.mean(error)}")


XYZ = vpm.particle_mesh.grid_positions

z_shape = XYZ.shape[3]
z_shapes = [z_shape // 2]
for idx_z in z_shapes:
    # Get the actual z value
    z = XYZ[2, 0, 0, idx_z]
    # Assert that the z value is the same for all the grid
    assert np.allclose(XYZ[2, :, :,idx_z], z)
    fig, ax = plt.subplots(1, 4, figsize=(15, 5))
    fig.suptitle(f"Z = {z}")

    plot = ax[0].pcolormesh(XYZ[0, :, :, idx_z], XYZ[1, :, :, idx_z], U_MESH[:, :, idx_z], cmap = 'turbo')
    fig.colorbar(plot, ax=ax[0])
    ax[0].set_title("Computed Velocity (VPM)")

    plot = ax[1].pcolormesh(XYZ[0, :, :, idx_z], XYZ[1, :, :, idx_z], U_OSEEN[:, :, idx_z], cmap = 'turbo') 
    fig.colorbar(plot, ax=ax[1])
    ax[1].set_title("Analytical Velocity")

    plot = ax[2].pcolormesh(XYZ[0, :, :, idx_z], XYZ[1,:, :, idx_z], error[:, :, idx_z], cmap = 'turbo')
    fig.colorbar(plot, ax=ax[2])
    ax[2].set_title("Error")
    
    # Error percentage
    error_percentage = np.abs(U_MESH - U_OSEEN) / np.abs(U_MESH)
    error_percentage[error_percentage > 0.9] = 0.0
    error_percentage = np.nan_to_num(error_percentage, posinf=0.0, neginf=0.0)
    plot = ax[3].pcolormesh(XYZ[0, :, :, idx_z], XYZ[1, :, :, idx_z], error_percentage[:, :, idx_z], cmap = 'turbo')
    fig.colorbar(plot, ax=ax[3])
    ax[3].set_title("Error percentage")

    plt.show()

In [ ]:
P_MESH = np.nan_to_num(vpm.particle_mesh.pressure, posinf=0.0, neginf=0.0)
P_OSEEN = np.nan_to_num(pressure_oseen, posinf=0.0, neginf=0.0)

# Get the error between the computed pressure and the hill vortex pressure
error = np.abs(P_MESH - P_OSEEN) 
print(f"Error: {np.max(error)}")
print(f"Error: {np.mean(error)}")

XYZ = vpm.particle_mesh.grid_positions
z_shape = XYZ.shape[3]
z_shapes = [z_shape // 2]
for idx_z in z_shapes:
    # Get the actual z value
    z = XYZ[2, 0, 0, idx_z]
    # Assert that the z value is the same for all the grid
    assert np.allclose(XYZ[2, :, :,idx_z], z)
    fig, ax = plt.subplots(1, 4, figsize=(15, 5))
    fig.suptitle(f"Z = {z}")

    plot = ax[0].pcolormesh(XYZ[0, :, :, idx_z], XYZ[1, :, :, idx_z], P_MESH[:, :, idx_z], cmap = 'turbo')
    fig.colorbar(plot, ax=ax[0])
    ax[0].set_title("Computed pressure (VPM)")

    plot = ax[1].pcolormesh(XYZ[0, :, :, idx_z], XYZ[1, :, :, idx_z], P_OSEEN[:, :, idx_z], cmap = 'turbo') 
    fig.colorbar(plot, ax=ax[1])
    ax[1].set_title("Analytical pressure")

    plot = ax[2].pcolormesh(XYZ[0, :, :, idx_z], XYZ[1,:, :, idx_z], error[:, :, idx_z], cmap = 'turbo')
    fig.colorbar(plot, ax=ax[2])
    ax[2].set_title("Error")

    # Error Percentage
    error_percentage = np.abs((P_MESH - P_OSEEN) / P_OSEEN) * 100
    error_percentage = np.nan_to_num(error_percentage, posinf=0.0, neginf=0.0)
    plot = ax[3].pcolormesh(XYZ[0, :, :, idx_z], XYZ[1,:, :, idx_z], error_percentage[:, :, idx_z], cmap = 'turbo')
    fig.colorbar(plot, ax=ax[3])
    ax[3].set_title("Error Percentage")
    plt.show()